In [18]:
import pandas as pd
import requests
from datetime import datetime, timedelta

In [19]:
def get_top_signals(count):
    df = pd.read_json(signals_path)
    past = datetime.now() - timedelta(days=30)
    past_str = past.strftime("%m/%d/%Y, %H:%M:%S")
    df_latest = df.loc[(df['date'] > past_str)]
    return df_latest.groupby('target_id')['session_id'].nunique().sort_values(ascending=False).head(count)

In [23]:
def query_top_movies(count=10):
    top_signals = get_top_signals(count)

    top_movies_query = ''
    for id in top_signals.index:
        if len(top_movies_query) > 0: top_movies_query += " "
        top_movies_query += '"' + str(id) + '"^' + str(top_signals[id])
    
    request = {
        "query": top_movies_query,
        "fields": ["id", "title", "origin", "release", "director", "cast"],
        "params": {
            "qf": "id",
            "hl": False,
            "mlt": False
        }
    }
    
    top_movies = requests.post(f"{url}/{collection}/search", json=request).json()
    return top_movies